In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%set_random_seed 12

# Offline RL theory

## Main issues

Can techniques from online RL, known for solving complex problems effectively, be applied to offline RL? 

<img src="_static/images/nb_94_on_policy_vs_off_policy.png" alt="offline_rl" style="width:60%">
<div class="slide title"> On-policy vs. off-policy approaches.
</div>

In off-policy online RL, we use a replay buffer to store $(state, action, reward)$ data, updating it as the learned policy improves. Why not apply an off-policy algorithm, filling the replay buffer directly with collected data.

<img src="_static/images/nb_94_off_policy_vs_offline.png" alt="offline_rl" style="width:70%">

**This is just a qualitative parallelism, and offline RL will work even if the data in your replay buffer is far from optimal.**

**However, even though both approaches seem similar, off-policy methods won't be able to work with collected data directly.**

**A bit of review:**

In particular many off-policy RL algorithms make use of the following approach:

$$
{\hat Q}^{k+1} \leftarrow L_1 = \arg \min_Q \mathbb{E}_{(s,a,s')\sim D} \left[\left( Q_\phi(s, a) - r(s, a) - \gamma \mathbb{E}_{a' \sim\pi_\theta(a'|s')}[Q_\phi(s', a')] \right)^2 \right]  \tag{Evaluation}
$$

$$
\pi_{k+1} \leftarrow L_2 = \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi_\theta(a|s)} Q^{k+1}_\phi(s, a) \right] \tag{Improvement}
$$

with:

$$ 
Q^\pi(s, a) = \mathbb{E}_\pi \left[ r_0 + \gamma r_1 + \gamma^2 r_2 + \ldots \mid s_0 = s, a_0 = a \right]
\tag{Q-value}
$$


where $D$ is the replay buffer, which in the offline RL case will be filled with the collected dataset.

**As seen in the (Evaluation) step, the only potential out-of-distribution (o.o.d) issue arises when computing action $a'$, as all other values ($s$, $a$, $s'$) are from the dataset $D$.**

<img src="_static/images/nb_94_q_value.png" alt="offline_rl" style="height:200px;">


If during the evaluation loop:

$$
{\hat Q}^{k+1} \leftarrow L_1 = \arg \min_Q \mathbb{E}_{(s,a,s')\sim D} \left[\left( Q_\phi(s, a) - r(s, a) - \gamma \mathbb{E}_{a' \sim\pi_\theta(a'|s')}[Q_\phi(s', a')] \right)^2 \right]  \tag{Evaluation}
$$

the policy, $\pi_\theta(.|s)$, samples o.o.d. actions like $a'_4$ in the figure, as we don't know the rewards on o.o.d. regions, $Q(s',a'_4)$, would be unpredictable.

<img src="_static/images/94_dqn_ood_case.png" alt="offline_rl" style="width:50%; display: block; margin-top: 0; padding-top: 0;">


If by chance, $Q(s',a'_4)$, is higher than that of in-distribution actions like, $a'_0, a'_1, a'_2$, this overestimated misinformation will propagate into the improvement step:

$$
\pi_{k+1} \leftarrow L_2 = \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi_\theta(a|s)} Q^{k+1}_\phi(s, a) \right] \tag{Improvement}
$$

as, during improvement, we try to find the policy that maximizes $Q(s,a)$. Therefore, the policy will have a tendency to go o.o.d. quite often.

**This could be catastrophic during inference!!**

The figure below illustrates the problem of overestimating Q-values in the off-policy Soft Actor Critic (SAC) algorithm on the half-cheetah environment.


<img src="_static/images/94_offpolicy_Q_values_overestimation.png" alt="offline_rl" style="width:70%">


**In online RL, overestimation is addressed through exploration, which is not possible in offline methods**

**Note: If we replace $a'$ with the dataset value in the evaluation step, we would learn $Q^\beta(s,a)$, the Q-value of the behavior policy. However, this would lead to the learned policy in the improvement step being generally far from optimal, especially when dealing with suboptimal data as is common in real scenarios.**


**The other important aspect of the offpolicy method introduced earlier is that it implements a dynamic programming approach that is responsible for implementing the "stitching" property, which is crucial for connecting suboptimal trajectories into optimal ones**.

<img src="_static/images/nb_94_stitching_property.png" alt="offline_rl" style="width:40%">

### Summary

- Off-policy methods are strong candidates for offline RL. They assume a replay buffer, and if they implement a dynamic programming approach, they are suited to combine pieces of different trajectories into a new, better one.

- However, they suffer from the overestimation problem when applied to data, which can cause the algorithm to easily go out of distribution.

**Let's go to the exercises in nb_95**

## Addressing out of distribution issues

### Overview

**The idea is to constraint the learned policy "to be close" to the behavioral one while still introducing a controlled distributional shift to enhance the policy without going out of distribution. This balancing act is a significant challenge and a focus of ongoing RL research.**

<img src="_static/images/nb_94_distributional_shift.png" alt="KL divergence" width=40%>

In other words, **you want the learned and behavioral policies to differ on $D$; for example, we don't want to exclude state-action pairs that may appear infrequently (few expert data) but could lead to higher-reward trajectories**, especially considering that these states may likely appear during inference (see figs. a-b).

<img src="_static/images/nb_94_ood_examples.png" alt="KL divergence" width=80%>

But at the same time, **we also want the policies to be close enough near to o.o.d. regions for the reasons already discussed** (fig-c).

**Note**: This type of control over missing and o.o.d. data is something that cannot be achieved with imitation learning.

To attain the aforementioned goals, offline RL algorithms can be classified into three primary categories:

**I - Policy constraint**

**II - Policy Regularization**

**III - Importance sampling**

### I - Policy constraint methods

One option we have to encourage to copy the expert knowledge in the data but restrict undesired o.o.d. situations is to constraint the learned policy to mimic the right behavior observed in the data.

#### a) Non-implicit or Direct

**We have access to the behavior policy, $\bf \pi_\beta$**


Since we already have $\pi_\beta$, we can constrain the learned and behavioral policy using:

\begin{equation}
D_{KL}(\pi(.|s)||\pi_{\beta}(.|s)) \leq \epsilon
\label{dk_1}
\end{equation}


<div style="margin-top: 20px;">
    <div style="display: flex; justify-content: space-between;">
        <div style="width: 49%;">
            <img src="_static/images/96_KL_divergence.png" alt="KL divergence" width="100%">
            <div class="slide title"> Fig.1: KL Divergence </div>
        </div>
        <div style="width: 10%;"></div> <!-- Empty div for space in the middle -->
        <div style="width: 40%;">
            <img src="_static/images/96_DKL_2.png" alt="Your Second Image" width="100%">
        </div>
    </div>
</div>

where the Kullback-Leibler divergence, $D_{KL}$, is defined as:

$$
D_{KL}(\pi(.|s)||\pi_{\beta}(.|s)) = \sum_a \pi(a|s) log \frac{\pi(a|s)}{\pi_{\beta}(a|s)} 
\label{dkl_2}
$$

<img src="_static/images/96_policy_constraint_DKL.png" alt="KL divergence" width=60%>
<div class="slide title"> Fig.2: DKL divergence's effect on out-of-distribution data </div>

**To summarize**: The $D_{KL}$ divergence discourages undesired o.o.d actions, as in case (b), yet encourages important o.o.d actions, as in case (a), crucial for enhancing the learned policy. This ensures that state distributions $d_{\pi}(s)$ and $d_{\pi_{\beta}}(s)$ are close around a given state $s$, so the distributional shift won't be too large. In contrast, case (b) will typically exhibits a large distributional shift. 

Moreover, actions such as $a_4$, in fig.2a, may guide our agent to highly rewarding regions, as illustrated in the figure below.

<img src="_static/images/96_critical_action_states.png" alt="KL divergence" width=50%>
<div class="slide title"> Fig.3: Critical actions may appear infrequently in the collected data but are crucial for finding the optimal policy. </div>

**These methods typically use the evaluation-improvement approach, introduced before, plus the $D_{KL}$ constraint**:

$$
{\hat Q}^{k+1} \leftarrow L_1 = \arg \min_Q \mathbb{E}_{(s,a,s')\sim D} \left[\left( Q_\phi(s, a) - r(s, a) - \gamma \mathbb{E}_{a' \sim\pi_\theta(a'|s')}[Q_\phi(s', a')] \right)^2 \right]  \tag{Evaluation}
$$

$$
\pi_{k+1} \leftarrow L_2 = \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi_\theta(a|s)} Q^{k+1}_\phi(s, a) \right] \tag{Improvement}
$$

$$
\begin{equation}
D_{KL}(\pi(.|s), \pi_{\beta}(.|s)) \leq \epsilon.
\tag{Constraint}
\end{equation}
$$


**Technical comment**: We could incorporate this constraint as a Lagrange multiplier, or sometimes it is absorbed in the evaluation and improvement steps:

$$
{\hat Q}^{\pi}_{k+1} \leftarrow \arg \min_Q \mathbb{E}_{(s,a,s')\sim D} \left[\left( Q(s, a) -  r(s, a) + \gamma \mathbb{E}_{a' \sim\pi_k(a'|s')}[{\hat Q}^{\pi}_k(s', a')] -\alpha D_{KL}(\pi_k(\cdot|s'), \pi_\beta(\cdot|s')) \right)^2 \right]
$$

$$
\pi_{k+1} \leftarrow \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi(a|s)} Q^{\hat{\pi}_{k+1}}(s, a) -\alpha D_{KL}(\pi_k(\cdot|s), \pi_\beta(\cdot|s)) \right] \\
$$

that produces similar results, with the advantage that is much easier to implement from a technical point of view.

**But what happens if we need to deviate considerably from the behavior policy, as can happens in realistic situations where the data collected is far from optimal?**. 

Let's  analize the simple example in the fig. below:

<img src="_static/images/96_support_policy_constraint.png" alt="offline_rl_4" width=70%>

where all data is in-distribution. Still the $D_{KL}$ constraint distribution will copy the bad behavior of $\pi_b$ .

What about constraining the policy support? In other words, **we don't focus on the probability of action $a$ within the dataset, but only on whether that action is included in the dataset**.

<img src="_static/images/policy_constraint_vs_support.png" alt="offline_rl_4" width=40%>
<div class="slide title"> Fig.4: distributional vs. support policy constraint </div>

**This is usually a good idea: Just constrain only allowed actions and let the algorithm find the action probabilities through the evaluation-improvement equations.**

**Summary** 

**Direct constraint methods assume you have access to the behavior policy.**

They are typically of two kinds:

- Constraint on distribution: easy to implement and generally effective, but they can sometime be too conservative.

- Constraint on support: a good choice when the behavior policy deviates significantly from optimality.

#### b) Implicit 

**We don't need $\pi_\beta$, and we can work directly with our data $D$**. Useful in complex high dimensional spaces where the BC of the behavioral policy is not accurate.

First we assume that we have $\pi_\beta$ (we will integrate it out later). **We maximize the difference reward**:

$$
\begin{align}
max_\pi \eta(\pi) &= max_\pi ( J(\pi) - J(\pi_\beta) ) \qquad \hbox{with} \quad J (\pi) = \mathbb{E}_{\tau \sim \pi}  \left[ \sum_{t = 0}^{\infty} \gamma^t r (s_t, a_t) \right] \\ \text{s.t.} \quad &D_{KL}(\pi(\cdot|s) || \pi_\beta(\cdot|s) ) \leq \epsilon
\tag{2}
\end{align}
$$

so as higher $\eta(\pi)$ the better $\pi$ . 

**Note**: This is a slightly different idea than what we usually follow to find the policy that maximizes the discounted reward.

It can be shown that (2) can be written as:

\begin{equation}
max_\pi \eta(\pi) = max_\pi \mathbb{E}_{s \sim d^{\pi}(s)} \mathbb{E}_{a \sim \pi(a|s)} [A^{\pi_\beta}(s, a)] \\ \text{s.t.} \quad D_{KL}(\pi(\cdot|s) || \pi_\beta(\cdot|s) ) \leq \epsilon
\tag{3}
\end{equation}


with $A^{\pi_\beta}(s, a) = Q^{\pi_\beta}(s, a) -V^{\pi_\beta}(s)$ the Advantage function.

As we will shown next, eq.3 will give us the optimal policy. 

Let's try to understand eq.3:

<img src="_static/images/96_difference_reward.png" alt="offline_rl_4" width=80%>
<div class="slide title"> Figure 5: Optimal policy from difference reward maximization.</div>

**In summary, eq.3 implies finding a policy $\pi(a|s)$ that generates state-action pairs $(s_0,a_0)$ (constrained through the $D_{KL}$ to be close to the dataset) that lead to trajectories in our dataset with maximum reward. In other words, we are trying to find the optimal policy within the data.**

Let's jump to the final solution of eq.3. If you are interested in more mathematical details see the slide below. The optimal policy, $\hat{\pi}_\theta (a|s)$, is:

$$
\hat{\pi}_\theta (a|s) \leftarrow
\arg\max_{\pi_\theta} \mathbb{\sum}_{(s,a)\sim D} \left[ \frac{1}{Z(s)} \log \pi_\theta(a|s) \exp\left(\frac{1}{\lambda} A^{D}(s, a)\right) \right]
\label{AWR}
\tag{4}
$$

**Interpretation: the maximization operation in eq.4 implies that $\pi_\theta(a|s)$ will be maximum in state-action pairs within or close to $D$, but only if $(s,a)$ brings the agent to a high reward region, i.e., if $\exp\left(\frac{1}{\lambda} A^{D}(s, a)\right)$ (or $A^{D}(s, a)$) is high.**

This is in perfect agreement with out previous interpretation in fig.5.

**Finally, note that to compute equation 3, we need $Q^{\pi_\beta}(s, a)$, so we could use the dynamic programming evaluation step introduced before, where eq.4, plays the role of the improvement step.**.

**ALERT!! : Only if you are interested in the math:**

It is easy to see that the solution to eq.3 is:

$
\pi^*(a|s) = \frac{1}{Z(s)} \pi_\beta(a|s) \exp\left(\frac{1}{\lambda} A^{\pi_\beta}(s, a)\right)  \quad \text{with } \lambda \in R
\tag{4}.
$

were $\lambda$ comes from a Lagrange multiplier that take into accounts the $D_{KL}$ constraint.

**Again eq.4 means that $\pi^*(a|s)$ is given by the probability of finding $(s,a)$ in the dataset (i.e. $\pi_\beta(a|s)$) times the probability that $(s,a)$ drives the agent to a high cumulative reward ($~ \exp\left(\frac{1}{\lambda} A^{\pi_\beta}(s, a)\right)$).**

Finally, if we approximate our theoretical optimal policy $\pi^*(a|s)$ by a DNN $\pi_\theta (a|s)$, we can use an SVI approach:

$$\begin{aligned}
\pi_\theta (a|s) &= \arg\min_{\pi_\theta} \mathbb{E}_{s \sim d_{\pi_\beta}(s)} \left[ D_{KL}(\pi^*(\cdot|s) \, \Vert \, \pi_\theta(\cdot|s)) \right] \\ &= \arg\max_{\pi_\theta} \mathbb{E}_{s \sim d_{\pi_\beta}(s)} \mathbb{E}_{a \sim \pi_\beta(a|s)} 
\left[ \frac{1}{Z(s)} \log \pi_\theta(a|s) \exp\left(\frac{1}{\lambda} A^{\pi_\beta}(s, a)\right) \right] 
\quad (5)
\end{aligned}$$

where the sampling is through $\pi_\beta$ that we can replace by the collected dataset:

$$
\pi_\theta (a|s) =
\arg\max_{\pi_\theta} \mathbb{\sum}_{(s,a)\sim D} \left[ \frac{1}{Z(s)} \log \pi_\theta(a|s) \exp\left(\frac{1}{\lambda} A^{D}(s, a)\right) \right]
\tag{6}
$$

#### Summary:

- Policy constraint methods are able to fill gaps in the collected data to guide the agent to high-reward regions while also discouraging o.o.d. behavior. This can be achieved by constraining the density or the support of the policy, if you have access to the behavior policy. If you don't, methods like implicit policy constraints could be the way to go.

- Policy constraint methods are powerful, but they can often be too pessimistic, which is undesirable.
  For example, if we know that a certain state has all actions yielding zero reward, we should not constrain the 
  policy in this state, as could affect our neural network approximator. 

**An alternative approach to avoid o.o.d actions without directly constraining the policies is to control o.o.d behavior from a Q-function perspective.**

### II - Policy Regularization methods

**This approach involves regularizing the value function directly, aiming to prevent overestimation, especially for actions that fall outside the distribution seen during training**

Advantages:

- **Applicable to different RL methods, including actor-critic and Q-learning methods.**

- **Doesn't necessitate explicit behavior policy modeling.**

<img src="_static/images/96_CQL_1.png" alt="offline_rl_4" width=100%>
<div class="slide title"> Fig.6: Policy regularization approach </div>


**Main idea: introduce a new policy $\mu(a|s)$ that attempts to find the actions $a$ maximizing the DNN $Q_\phi$, while simultaneously minimizing $Q_\phi$ within the $\phi$ parameter space. This effect is particularly crucial for o.o.d. actions, which are generally overestimated, as previously observed.**

**The policy $\mu$ doesn't necessarily have to be proportional to $\pi(a|s)$, but it should aim to maximize Q(s,a).**

These are some popular offline RL algorithms, but the list is longer:

<img src="_static/images/nb_94_popular_offline_rl_algorithms.png" alt="offline_rl_4" width=100%>

It's worth noting that most of these algorithms fall into the categories we've already discussed, namely policy constraint and policy optimization methods

# Appendix: Short review of some popular offline RL algorithms

## Introduction

In this notebook, we will explore several key algorithms that aim to address distributional shift issues within offline reinforcement learning. It's worth noting that the field of offline RL is evolving rapidly, and this list is by no means exhaustive. Many of the concepts and strategies employed by these algorithms find applications and improvements in various other approaches.

A common approach followed by many algorithms in offline RL involves an actor-critic methodology. Within this framework, there is an iterative process of evaluation and improvement, characterized by:

$$
\begin{equation}
{\hat Q}^{\pi}_{k+1} \gets \arg \min_Q \mathbb{E}_{s,a \sim \mathcal{D}} \Big[\big(Q(s,a) - \mathcal{B}^{\pi}_k Q(s,a)\big)^2\Big].
\tag{Evaluation}
\end{equation}
$$

$$
\begin{equation}
\mathcal{B}^{\pi}Q = r + {\gamma}\mathbb{E}_{s' \sim D, a' \sim \pi}Q(s',a') 
\tag{Bellman backup op.}
\end{equation}
$$


$$
\begin{equation}
\pi_{k+1} \leftarrow \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi(a|s)} Q^{\hat{\pi}}_{k+1}(s, a) \right] \tag{Improvement}
\end{equation}
$$



So the main idea is to modify the Evaluation/Improvement steps to improve the distributional shift problems.

## Batch Constrained deep Q-learning (BCQ) algorithm

The main idea is pictured in the figure below.

<img src="_static/images/97_BCQ_algo_1.png" alt="offline_rl_4" width=200%>
<div class="slide title"> Fig.5: BCQ approach to offline RL </div>

In BCQ, the policies $\pi$ and $\pi_\beta$ are not constrained by the $D_{KL}$ divergence, but we still ensure that $\pi(s)$ generates similar actions to $\pi_\beta(s)$ through a generative model, in this case, a Variational Autoencoder (VAE), $G_\omega$: 

$$
\pi(s) = \arg\max_{a_i} Q_\theta(s, a_i),
\\ \{a_i \sim G_\omega(s)\}_{i=1}^n
\tag{7}
$$

Therefore, this method falls under the direct policy constraint approach discussed earlier.

The BCQ algorithm uses a clipped Double Deep Q-Learning (clipped-DDQ) approach to compute the Q-values:

$$
L(\theta_i, D) = \mathbb{E}_{ s,a,r,s' \sim D} \left[  Q_{\theta_i}(s,a) - y(r,s') \right]
$$

with

$$
y(r,s') = r + \gamma max_{a_i} min_{i=1,2} Q_{\theta_i, targ} (s', a_i)
$$

The minimum is taken to avoid the overestimation of Q-values, an issue that also occurs in these kinds of methods in online RL. In offline RL, as we saw, o.o.d. actions are the ones that typically produce such overestimations. Therefore, clipped-DDQ also introduces control over this issue at the Q-value level, achieving a similar effect to what policy regularization methods aim for with a lower bound on Q-values.

**A few technical details**:

The actions in eq. 7 are clipped with some noise $\epsilon$, hence the name clipped, as this also helps to avoid overestimation of Q-values.

$$
a \rightarrow clip [a + clip(\epsilon, -c, c), a_{low}, a_{high}]
$$

We allow actions with high Q-values to introduce some uncertainty, helping the algorithm explore regions of lower reward to avoid overestimation effects.

Finally, as running a VAE during training can be computationally expensive, the algorithm introduces a perturbation model $\xi_\phi(s, a_i, \Phi)$, which outputs an adjustment to an action $a$ in the range $[-\Phi, \Phi]$. Therefore, eq.7 becomes:

$$
\pi(s) = \arg\max_{a_i + \xi_\phi(s, a_i, \Phi)} Q_\theta(s, a_i + \xi_\phi(s, a_i, \Phi)),
\\ \{a_i \sim G_\omega(s)\}_{i=1}^n
$$


Note that if $\Phi=0$ and $n=1$ the policy will resemble behavioral cloning.
On the opposite side if d $\Phi \rightarrow a_{max} - a_{min}$ and $n \rightarrow \infty$, then the algorithm approaches Q-learning, as the policy begins to greedily maximize the value function over the entire action space.

**Pros**: As it learns how to generate new actions not included in the dataset, it is suitable for small datasets and for unbalanced sets where a few unrepresented actions could be important for the task to be solved.

**Cons**:  Since BCQ generates actions from a VAE, if the dataset used to train it underrepresents some important actions, the VAE may struggle to generate meaningful actions around those states, making the discovery of new or unconventional actions difficult. This is one of the limitations of constrained policy approaches.

## Conservative Q-Learning (CQL) algorithm

CQL follows a pessimistic approach by considering a lower bound of the Q-value. In the paper the authors shown that the solution of:

$\hat{Q}^{k+1}_{\text{CQL}} \gets \hbox{argmin}_Q \left[ \color{red} {\alpha\big(\mathbb{E}_{s \sim \mathcal{D}, a \sim \mu}[Q(s,a)] - \mathbb{E}_{s,a \sim \mathcal{D}}[Q(s,a)]\big)} + \frac{1}{2} \mathbb{E}_{s,a \sim \mathcal{D}} \Big[\big(Q(s,a) - \mathcal{B}^{\pi}Q(s,a)\big)^2\Big] \right].$

for $\mu = \pi$ is a lower bound for the Q value.

The nice thing about this method is that it can be applied to any Actor Critic method in a few lines of code.

CQL Focuses on **conservative value estimation** to provide lower bounds on the expected return of a policy. Aims to reduce overestimation bias and ensure that the policy remains within a safe region of the state-action space. Achieves safe exploration by constructing action sets that cover a broader range of state-action pairs. 
Well suited for scenarios where safety is a top priority, as it **reduces the risk of catastrophic actions**.

Note that BCQ could be better to discover novel actions and to use the collected data more efficiently but may not guarantee complete safety!.

## Implicit Q-Learning (IQL) algorithm

This is another clever idea to avoid going out of distribution. Let's revisit the ideas for evaluation improvement, assuming that we only operate with state-action pairs from the dataset in a SARSA-style approach, i.e.:

$$
{\hat Q}_{k+1} \leftarrow \arg \min_Q \mathbb{E}_{(s,a,s',a')\sim D} \left[\left( Q(s, a) -  r(s, a) + \gamma{\hat Q}_k(s', a')  \right)^2 \right]
$$

$$
\pi_{k+1} \leftarrow \arg \max_{\pi} \mathbb{E}_{s\sim D} \left[ \mathbb{E}_{a \sim\pi(a|s)} Q^{\hat{\pi}_{k+1}}(s, a)  \right] \\
$$

This is indeed a valid approach. It's important to note that running the evaluation-improvement loop makes sense only once. During evaluation, we compute the $Q$-values of the behavior policy and derive the optimal policy based on those $Q$-values in the improvement step. Further iterations would be futile since we are limited to the fixed dataset.

However, this idea often falls short in finding an optimal policy for many real-world problems. Intuitively, if our data is suboptimal, the Q-values derived from that data will also be suboptimal.

The core principle of IQL is to utilize a pessimistic Q-value lower bound during evaluation, similar to policy regularization, while also ensuring consistency with in-distribution data. This strategy enables a multi-step process, facilitating multiple evaluation-improvement iterations. With each iteration, a new estimate for Q(s,a) is derived, encouraging a deeper exploration of the Q-functions and enabling the capture of broader correlations.

<img src="_static/images/96_one_step_vs_multiple_steps.png" alt="offline_rl_4" width=80%>
<div class="slide title"> Fig.6: one vs multiple step approaches.  </div>


These are the main steps involved in the IQL approach:

$$L_V(\psi) = E_{(s,a)\sim D}[L_2^{\tau}(Q_{\hat{\theta}}(s, a) - V_{\psi}(s))]$$

$$L_Q(\theta) = E_{(s,a,s') \sim D}\left[(r(s, a) + \gamma V_{\psi}(s') - Q_{\theta}(s, a))^2\right]$$

and for the policy improvement step, it uses an advantage weighted regression:

$$L_\pi(\phi) = E_{(s,a)\sim D} \left[\exp(\beta(Q_{\hat{\theta}}(s, a) - V_{\psi}(s))) \log \pi_{\phi}(a|s)\right]
$$

similar to eq.4 . The lower bound used here is the 'expectile' shown in the figure below.


<img src="_static/images/96_expectile.png" alt="offline_rl_4" width=80%>
<div class="slide title"> Fig.7: Expectile of a two dimenstional random variable.  </div>



## Q-Transformer

Adapts the bellman equations to deal with a transformer architecture: [paper](https://qtransformer.github.io/) and [blog](https://qtransformer.github.io/)

# References

[Schulman et al. 2017 - Trust Region Policy Optimization](https://arxiv.org/pdf/1502.05477.pdf)

[Kumar et al. 2020 - Conservative Q-Learning for Offline Reinforcement Learning](https://arxiv.org/pdf/2006.04779.pdf)

[ Levine et al. 2021 - Offline Reinforcement Learning: Tutorial, Review,
and Perspectives on Open Problems ](https://arxiv.org/pdf/2005.01643.pdf)

[Peng et al. 2019 - Simple and Scalable Off-Policy Reinforcement Learning](https://arxiv.org/abs/1910.00177)

[Nair et al. '2020 - AWAC: Accelerating Online Reinforcement Learning with Offline Datasets](https://arxiv.org/abs/2006.09359)
